# Worldwide Total Video Game Console Sales Analysis

## 1. Introduction.

This notebook covers the analysis of some of the worlds best selling video games consoles, ranging from the 1970's to current day consoles.

## 2. Requirements.

The main insights that are required from this analysis are:

- Rank the consoles by their overall sales.
- Rank the consoles by their regional sales.
- Rank the consoles by their overall number of games.
- Rank the consoles by their overall msrp (highest to lowest).
- Rank the consoles by their form factor (handheld or not) and overall sales.
- Rank the consoles by their form factor (handheld or not) and regional sales.
- Rank the vendors by the overall consoles sold and what each one was.
- Rank the vendors by the regional consoles sold and what each one was.
- Determine what the most popular console was for each decade that is present in the data.
- The average price of a console for each decade in the data.
- The average life cycle of a console overall and by decade.
- The overall failure rate number for each console.

## 3. Data Collection.

The bulk of the data for this was collected using a web scraper from https://www.videogameconsolelibrary.com.

The originally scraped data was saved to a CSV file called 01-original-data.csv, which is stored in the data folder.

Additionally, a number of features were added to the data using Excel from a number of sources. These sources include:

- https://www.computinghistory.org.uk/ - Used for additional stats, such as launch price and release / discontinuation dates.
- https://www.macrotrends.net/ - Used for currency conversions of older systems reported in non-USD currencies.
- https://necretro.org/ - Used for additional stats, such as launch price and release / discontinuation dates.
- https://en.wikipedia.org/ Used for additional stats, such as launch price and release / discontinuation dates.

Lastly, the failure rate information for the xBox 360, PlayStation 3 and Wii were obtained from the below link:
https://www.squaretrade.com/htm/pdf/SquareTrade_Xbox360_PS3_Wii_Reliability_0809.pdf

The failure rate for other consoles is unknown so an estimate of 7.5% was used. This was determined by taking the failure rate of the Nintendo Wii (2.7%), which was rounded to 3% and the failure rate of the PlayStation 3 (10%) and working out the middle point between them (7.5%).

The final data that will be used for this is stored in `./data/02-processed-data.xlsx`. A CSV version (`02-processed-data.csv`) is also stored in the same location.

**NOTE** The Pandora console was removed as it is used mostly for emulation and has no real native games.

## 4. Data Dictionary.

The data dictionary is stored in `./data/00-data-dictionary.xlsx`.

## 5. Data Importing And Formatting.

### 5.1. Import The Required Libraries / Modules.

In [ ]:
# -- Required libraries:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# -- Required modules:
from modules.seaborn_templates import colours, sns_barplot_ax

### 5.2. Import The Data.

In [ ]:
# -- Select the columns to use:  
columns = ["vendor", "platform", "handheld", "released_date", "discontinued_date",
           "total_available_games", "failure_rate_percentage", "msrp_usd",
           "global_millions", "europe_millions", "north_america_millions", 
           "japan_millions", "rest_of_the_world_millions", 
           "unknown_sales_millions"]

df = pd.read_excel(io = "./data/02-processed-data.xlsx", 
                   sheet_name = "02-processed-data",
                   usecols = columns,
                   dtype = {"global_millions": np.float64,
                            "europe_millions": np.float64, 
                            "north_america_millions": np.float64,
                            "japan_millions": np.float64, 
                            "rest_of_the_world_millions": np.float64,
                            "unknown_sales_millions": np.float64})

# -- Reindex the dataframe so that the columns are in the required order:
df = df.reindex(columns = columns)

### 5.3. Set Colour Pallette(s).

In [ ]:
# -- Create the default colour pallette:
colour_pallette = colours(data = df)

In [ ]:
# -- Set the defaults for Seaborn.
# -- Set the default size of figures:
sns.set(rc = {"figure.figsize": (8, 8)})


# -- Set the background colour of the axes and turn on the legend frame:
sns.set_style(rc = {'figure.facecolor': 'white',
                    'axes.labelcolor': '.15',
                    'xtick.direction': 'out',
                    'ytick.direction': 'out',
                    'xtick.color': '.15',
                    'ytick.color': '.15',
                    'axes.axisbelow': True,
                    'grid.linestyle': '-',
                    'text.color': '.15',
                    'font.family': ['sans-serif'],
                    'font.sans-serif': ['Arial',
                                        'DejaVu Sans',
                                        'Liberation Sans',
                                        'Bitstream Vera Sans',
                                        'sans-serif'],
                    'lines.solid_capstyle': 'round',
                    'patch.edgecolor': 'w',
                    'patch.force_edgecolor': True,
                    'image.cmap': 'rocket',
                    'xtick.top': False,
                    'ytick.right': False,
                    'axes.grid': True,
                    'axes.facecolor': '#EAEAF2',
                    'axes.edgecolor': 'black',
                    'grid.color': 'white',
                    'axes.spines.left': True,
                    'axes.spines.bottom': True,
                    'axes.spines.right': False,
                    'axes.spines.top': False,
                    'xtick.bottom': True,
                    'ytick.left': False,
                    "legend.frameon": True})

### 5.4. Check The DataTypes.

In [ ]:
df.dtypes

### 5.5. Check The First Five Rows.

In [ ]:
df.head(n = 5)

### 5.6. Check For NaN Values.

In [ ]:
df.isna().sum()

Observation: There are no NaN (not a number) values in the dataframe. 

As a result, it is ok to start exploring the data.

## 6. Exploratory Data Analysis.

### 6.1. Rank Consoles By Global Sales.

#### 6.1.1. Filter Data.

**NOTE:** Limiting results to top 20 for ease of output.

In [ ]:
global_sales = df[["vendor","platform", "global_millions"]]\
                   .sort_values(by = "global_millions", 
                                ascending = False).head(20)\
                   .reset_index(drop = True)

#### 6.1.2. Show Global Sales (Table).

In [ ]:
global_sales.head(20)

#### 6.1.3. Show Global Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = global_sales,
               x_axis = "global_millions",
               x_axis_label = "Global (Millions Sold)",
               x_axis_max_value = float(global_sales.global_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (Globally)");

#### 6.1.4. Observations.

From the results of the date, the following has been observed:
- The top 5 selling consoles (globally) are dominated by Sony and Nintendo.
  - Both of the companies are Japanese based.
- Of the top 10 selling consoles (globally), only one is not made by Sony or Nintendo.
  - That is the ninth spot that is taken by Microsoft with the Xbox 360.
- There are a total of five vendors in the top 20 list, with Sega (17th) and Atari (19th) having one entry each. The remainder consists of Sony, Nintendo and Microsoft.

### 6.2. Rank Consoles By Regional Sales.

#### 6.2.1. European Sales.

##### 6.2.1.1. Filter Data.

In [ ]:
european_sales = df[["vendor","platform", "europe_millions"]]\
                     .sort_values(by = "europe_millions", 
                                  ascending = False).head(20)\
                     .reset_index(drop = True)

##### 6.2.1.2. Show European Sales (Table).

In [ ]:
european_sales.head(20)

##### 6.2.1.3. Show European Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = european_sales,
               x_axis = "europe_millions",
               x_axis_label = "Europe (Millions Sold)",
               x_axis_max_value = float(european_sales.europe_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (Europe)");

##### 6.2.1.4. Observations.

#### 6.2.2. Japanese Sales.

##### 6.2.2.1. Filter Data.

In [ ]:
japanese_sales = df[["vendor","platform", "japan_millions"]]\
                     .sort_values(by = "japan_millions", 
                                  ascending = False).head(20)\
                     .reset_index(drop = True)

##### 6.2.2.2. Show Japanese Sales (Table).

In [ ]:
japanese_sales.head(20)

##### 6.2.2.3. Show Japanese Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = japanese_sales,
               x_axis = "japan_millions",
               x_axis_label = "Japan (Millions Sold)",
               x_axis_max_value = float(japanese_sales.japan_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (Japan)");

##### 6.2.2.4. Observations.

#### 6.2.3. North American Sales.

##### 6.2.3.1. Filter Data.

In [ ]:
na_sales = df[["vendor","platform", "north_america_millions"]]\
               .sort_values(by = "north_america_millions", 
                            ascending = False).head(20)\
               .reset_index(drop = True)

##### 6.2.3.2. Show North American Sales (Table).

In [ ]:
na_sales.head(20)

##### 6.2.3.3. Show North American Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = na_sales,
               x_axis = "north_america_millions",
               x_axis_label = "North America (Millions Sold)",
               x_axis_max_value = float(na_sales.north_america_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (North America)");

##### 6.2.3.4. Observations.

#### 6.2.4. Rest Of The World Sales.

##### 6.2.4.1 Filter Data.

In [ ]:
row_sales = df[["vendor","platform", "rest_of_the_world_millions"]]\
                .sort_values(by = "rest_of_the_world_millions", 
                             ascending = False).head(20)\
                .reset_index(drop = True)

##### 6.2.4.2. Show Rest Of The World Sales (Table).

In [ ]:
row_sales.head(20)

##### 6.2.4.3. Show Rest Of The World Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = row_sales,
               x_axis = "rest_of_the_world_millions",
               x_axis_label = "Rest Of The World (Millions Sold)",
               x_axis_max_value = float(row_sales.rest_of_the_world_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (Rest Of The World)");

##### 6.2.4.4. Observations.

#### 6.2.5. Uncategorised Sales.

##### 6.2.5.1 Filter Data.

In [ ]:
unknown_sales = df[["vendor","platform", "unknown_sales_millions"]]\
                .sort_values(by = "unknown_sales_millions", 
                             ascending = False).head(20)\
                .reset_index(drop = True)

##### 6.2.5.2. Show Uncategorised Sales (Table).

In [ ]:
unknown_sales.head(20)

##### 6.2.5.3. Show Uncategorised Sales (Bar Chart).

In [ ]:
sns_barplot_ax(data = unknown_sales,
               x_axis = "unknown_sales_millions",
               x_axis_label = "Unknown (Millions Sold)",
               x_axis_max_value = float(unknown_sales.unknown_sales_millions.max()),
               y_axis = "platform",
               y_axis_label = "Console",
               hue = "vendor",
               pallette = colour_pallette,
               title = "Top 20 Selling Consoles (Unknown Sales)");

##### 6.2.5.4. Observations.

### 6.3. Rank Consoles By Total Games Available.

#### 6.3.1. Filter Data.

In [ ]:
# -- Create a dataframe with the required filters and features:
games_ranking = df[["vendor","platform", "total_available_games", "released_date"]]\
                 .sort_values(by = "total_available_games", 
                              ascending = False)\
                 .reset_index(drop = True)

# -- Add 1 to each index to give the actual rank rather than starting at 0:
games_ranking.index = games_ranking.index +1

#### 6.3.2. Show Top 20 Consoles By Available Games.

In [ ]:
games_ranking.head(20)

#### 6.3.3. Show Bottom 20 Consoles By Available Games.

In [ ]:
games_ranking.tail(20)

#### 6.3.4. Observations.

- The top 20 consoles with the most games are again dominated by Nintendo, Microsoft and Sony, along with Sega.
- With the exception of one (Nintendo), the bottom 20 consoles by available games are from vendors that don't make their own consoles anymore.

### 6.4. Rank Consoles By MSRP (Highest To Lowest).

#### 6.4.1. Filter Data.

In [ ]:
# -- Create a dataframe with the required filters and features:
msrp_ranking = df[["vendor","platform", "msrp_usd", "released_date"]]\
                 .sort_values(by = "msrp_usd", 
                              ascending = False).head(20)\
                 .reset_index(drop = True)

# -- Add 1 to each index to give the actual rank rather than starting at 0:
msrp_ranking.index = msrp_ranking.index +1

#### 6.4.2. Show Results (Table).

In [ ]:
msrp_ranking

#### 6.4.3. Observations.

- Of the top 20 list, only Microsoft, Nintendo and Sony are still manufacturing consoles.
- The majority of the top 20 highest priced consoles at launch are from companies that no longer manufacture consoles under their respective name.
- The top 5 most expensive consoles were released between 1990 and 1999.
- Of the top 20, 10 were released between 1990 and 1999.

### 6.5. Rank Consoles By Form Factor (Handheld Or Not) Base On World-Wide Sales.

### 6.6. Rank Consoles By Form Factor (Handheld Or Not) Base On Regional Sales.

### 6.7. Rank Vendors By World-Wide Consoles Sold And What Each Console Is.

### 6.8. Rank Vendors By Regional Consoles Sold And What Each Console Is.

### 6.9. What Is The Most Popular Console For Each Decade.

### 6.10. The Average Price Of A Console For Each Decade.

### 6.11. The Average Life Cycle Of A Console Overall And By Decade.

### 6.12. The Failure Rate For Each Console.